In [126]:
from sympy import *
from IPython.display import display, Math, Latex, Markdown

In [127]:
#init_printing()

Resulting light = light coming from pixel * transmittance + integral over all points inbetween of (light coming from that point * transmittance from that point to camera)

Cryteks "exponential height fog" is used to simply scale the constants $\sigma_{a,c}$ and $\sigma_{s,c}$ depending on the height:

(Reformulated to divide by c instead. That way increasing c increases fade (more intuitive))

In [128]:
c = symbols('c',nonzero=True, infinite=False, real=True)

In [129]:
sigma_ac, sigma_sc = symbols('sigma_ac sigma_sc', real=True, positive=True)

In [130]:
def sigma_a(y):
    return sigma_ac*exp(-y/c)
def sigma_s(y):
    return sigma_sc*exp(-y/c)

In [131]:
def sigma_t(y):
    return sigma_a(y) + sigma_s(y)

Transmittance from pixel to camera:\
camera: $o$, pixel: $o+D*d$\
(in theory $o, d$ are vector quantities, but only the y component matters. so theyre treated as scalars in the following)

In [132]:
o, t, d, D = symbols('o t d D',infinite=False)

In [133]:
i = Integral(sigma_t(o+t*d), (t, 0, D))

In [134]:
exp(-i)

exp(-Integral(sigma_ac*exp((-d*t - o)/c) + sigma_sc*exp((-d*t - o)/c), (t, 0, D)))

Solving the integral

Simple in the case that d = 0

In [135]:
i0 = Integral(sigma_t(o), (t, 0, D))
i0

Integral(sigma_ac*exp(-o/c) + sigma_sc*exp(-o/c), (t, 0, D))

In [136]:
i0 = sigma_t(o)*Integral(1, (t, 0, D))
i0

(sigma_ac*exp(-o/c) + sigma_sc*exp(-o/c))*Integral(1, (t, 0, D))

In [137]:
simplify(i0)

D*(sigma_ac + sigma_sc)*exp(-o/c)

otherwise

In [138]:
d = symbols('d',nonzero=True, infinite=False)
i = Integral(sigma_t(o+t*d), (t, 0, D))
i

Integral(sigma_ac*exp((-d*t - o)/c) + sigma_sc*exp((-d*t - o)/c), (t, 0, D))

In [139]:
i.doit()

(-c*sigma_ac - c*sigma_sc)*exp((-D*d - o)/c)/d - (-c*sigma_ac - c*sigma_sc)*exp(-o/c)/d

(Could be compacted, but that form lead to numerical issues)

Next the 2nd part of the sum has to be calculated: Integrating the light reaching the camera from every point between the camera and the pixel

Again, first calculating the result for the case that $d=0$:

In [144]:
d = 0

In [145]:
def transmittanceIntegral(o,d,D):
    # return (-c*sigma_ac - c*sigma_sc)*exp((-D*d-o)/c)/d - (-c*sigma_ac - c*sigma_sc)*exp(-o/c)/d
    return D*(sigma_ac+sigma_sc)*exp(-o/c)

In [146]:
l_i, t_2 = symbols('l_i t_2')

In [147]:
#Helper function to calculate transmittance from the origin along a direction up until a given depth
def transmittance(o, d, D):
    return exp(-transmittanceIntegral(o,d,D))

In reality we'd have to solve the following integral to determine $L_s$ since $L_i$ most likely isnt uniform, but for now we assume that its constant. In that case were only left with the integral over the phase function which by definition is 1

In [149]:

display(Math('\int_{S^2} \mathrm{p}(\omega_i,-\omega)L_i(o+t*d,\omega_i)\mathrm{d}\omega_i'))


<IPython.core.display.Math object>

In [148]:
def L_s(y): #in scatterd light at height
    return sigma_s(y)*l_i

In [150]:
i0 = Integral(transmittance(o, d, t) * L_s(o+t*d), (t, 0, D))
i0

Integral(l_i*sigma_sc*exp(-o/c)*exp(-t*(sigma_ac + sigma_sc)*exp(-o/c)), (t, 0, D))

In [151]:
i0 = i0.doit()
i0

l_i*sigma_sc/(sigma_ac + sigma_sc) - l_i*sigma_sc*exp(-D*(sigma_ac + sigma_sc)*exp(-o/c))/(sigma_ac + sigma_sc)

The general case:

In [152]:
d = symbols('d',nonzero=True, real=True)

In [153]:
def transmittanceIntegral(o,d,D):
    return (-c*sigma_ac - c*sigma_sc)*exp((-D*d-o)/c)/d - (-c*sigma_ac - c*sigma_sc)*exp(-o/c)/d
    # return D*(sigma_ac+sigma_sc)*exp(-o/c)

In [154]:
i = Integral(transmittance(o, d, t) * L_s(o+t*d), (t, 0, D))
i

Integral(l_i*sigma_sc*exp((-d*t - o)/c)*exp(-(-c*sigma_ac - c*sigma_sc)*exp((-d*t - o)/c)/d + (-c*sigma_ac - c*sigma_sc)*exp(-o/c)/d), (t, 0, D))

In [155]:
i = i.doit()
i

-l_i*sigma_sc*exp(-(-c*sigma_ac - c*sigma_sc)*exp((-D*d - o)/c)/d + (-c*sigma_ac - c*sigma_sc)*exp(-o/c)/d)/(sigma_ac + sigma_sc) + l_i*sigma_sc/(sigma_ac + sigma_sc)

Comparing both results:

In [156]:
i0

l_i*sigma_sc/(sigma_ac + sigma_sc) - l_i*sigma_sc*exp(-D*(sigma_ac + sigma_sc)*exp(-o/c))/(sigma_ac + sigma_sc)

In [157]:
i

-l_i*sigma_sc*exp(-(-c*sigma_ac - c*sigma_sc)*exp((-D*d - o)/c)/d + (-c*sigma_ac - c*sigma_sc)*exp(-o/c)/d)/(sigma_ac + sigma_sc) + l_i*sigma_sc/(sigma_ac + sigma_sc)

It becomes obvious that the result in both cases is simply:

In [158]:
T_r = symbols('T_r')
l_i*sigma_sc/(sigma_ac+sigma_sc) - l_i*sigma_sc*T_r/(sigma_ac+sigma_sc)

-T_r*l_i*sigma_sc/(sigma_ac + sigma_sc) + l_i*sigma_sc/(sigma_ac + sigma_sc)

Where $T_r$ is the total transmittance from the camera to the pixel, and $\frac{\sigma_{sc}}{\sigma_{ac}+\sigma_{sc}}$ is the volumetric albedo

This formulation also explains the widely used fog approximation:\
$PixelColor * Transmittance + FogColor * (1-Transmittace)$\
Since setting $\sigma_{ac}$ to 0 (meaning no absorption, and only scattering is happening) results in:\
$(1-T_r)*l_i$